In [1]:
import synergy_scraper as ss

In [12]:
feats = ["Player", "GP", "Poss", "USG%", "PPP", "PPG", "RPG", "APG", "SPG", "BPG", "TOVPG", "FTA PG", "FTM PG", "FGA PG", "FGM PG", "3PA PG", "3PM PG", "Ast/TO", "Ast Per 100 Poss", "Stl Per 100 Poss", "Blk Per 100 Poss", "Reb Per 100 Poss", "Off Reb Per 100 Poss", "Def Reb Per 100 Poss", "TO%", "FT%", "FG%", "3 FG%", "eFG%"]

In [13]:
bs = ss.CumulativeBoxScore("boxscore.html")
df = bs.df()[feats]
df.to_csv("caltech.csv")

In [5]:
bs = ss.CumulativeBoxScore("wcb.html")
df = bs.df()[feats]
df.to_csv("wcb.csv")

In [4]:
import pandas as pd

In [20]:
import os
class TeamBoxScore:
  def __init__(self, team_folder, opponent_folder):
    self.team_folder = team_folder
    self.opponent_folder = opponent_folder

  def team_box_scores(self, home=False):
    totals = {"DREB": 0, "OREB": 0, "PTS": 0, "POSS": 0}

    if home:
      folder = self.team_folder
    else:
      folder = self.opponent_folder

    for filename in os.listdir(folder):
      f = os.path.join(folder, filename)
      html = pd.read_html(f)
      starters = html[0].iloc[[-1]]
      bench = html[1].iloc[[-1]]

      if filename=="game4.html" and home == False:
        print(starters["Pts"].values[0]/starters["PPP"].values[0])
      
      totals["DREB"] += starters["Def Reb"].values[0] + bench["Def Reb"].values[0]
      totals["OREB"] += starters["Off Reb"].values[0] + bench["Off Reb"].values[0]
      totals["PTS"] += starters["Pts"].values[0] + bench["Pts"].values[0]
      totals["POSS"] += starters["Pts"].values[0]/float(starters["PPP"].values[0]) + bench["Pts"].values[0]/float(bench["PPP"].values[0])

    return totals 

  def get_rates(self):
    team_totals = self.team_box_scores(home=True)
    opp_totals = self.team_box_scores(home=False)

    team_rates = {}
    team_rates["DREB"] = team_totals["DREB"] / (team_totals["DREB"] + opp_totals["OREB"])
    team_rates["OREB"] = team_totals["OREB"] / (team_totals["OREB"] + opp_totals["DREB"])
    team_rates["ORTG"] = team_totals["PTS"] / team_totals["POSS"] * 100
    team_rates["DRTG"] = opp_totals["PTS"] / opp_totals["POSS"] * 100

    return team_rates

In [21]:
TeamBoxScore("caltech", "opponent").get_rates()

63.9686684073107


{'DREB': 0.7869565217391304,
 'OREB': 0.3179916317991632,
 'ORTG': 72.42539511708293,
 'DRTG': 98.39971466141044}